In [1]:
import random
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import gym as gym
from gym import spaces
import carla
import numpy as np
from collections import deque
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
import csv
from time import sleep

2023-04-30 23:53:52.314667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 23:53:52.605123: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-30 23:53:53.676536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kololhe/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib
2023-04-30 23

In [2]:
threshold = 25
CSV_FILE = '/home/kololhe/stuff/waypoint_with_xy.csv'


In [3]:
def read_waypoints(file_path):
    waypoints = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            v, yaw_c, yaw_n, r, x, y = map(float, row)
            waypoints.append((v, yaw_c, yaw_n, r, x, y))
            
    return waypoints  


def setupCarla():

    # Connect to CARLA server
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)

    return client.get_world()


In [4]:
class CarlaInstance(gym.Env):
    metadata = {}

    def __init__(self, world, waypoints):
        #super(CarlaInstance, self).__init__()
        try:
            self.reached_waypoint_list = []
            self.waypoints = waypoints
            self.current_waypoint_index = 0
            

            self.blueprint_library = world.get_blueprint_library()
            world.set_weather(carla.WeatherParameters.ClearNoon)


            # Spawn vehicle
            vehicle_bp = random.choice(self.blueprint_library.filter('wrangler_rubicon'))
            spawn_point = carla.Transform(carla.Location(x=-23.6,y=137.5,z=1),carla.Rotation(yaw=0))
            self.vehicle = world.spawn_actor(vehicle_bp, spawn_point)
            #self.vehicle.set_simulate_physics(True)
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0, steer=0.0))
            sleep(2.0)

            # Attach Lane Invasion Sensor to car
            sensor_bp = world.get_blueprint_library().find('sensor.other.lane_invasion')
            sensor_transform = carla.Transform(carla.Location(x=2.5, z=0.7))
            self.lane_invasion_sensor = world.spawn_actor(sensor_bp, sensor_transform, attach_to=self.vehicle)
            self.lane_invasion_sensor.listen(lambda event: self.on_lane_invasion(event))

            self.collision_sensor = world.spawn_actor(world.get_blueprint_library().find('sensor.other.collision'), sensor_transform, attach_to=self.vehicle)
            self.collision_sensor.listen(lambda event: self.on_lane_invasion(event))

            self.action_space = spaces.MultiDiscrete([11, 21, 11])
            self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(40, ), dtype=np.float64)
            self.problem = bool
            self.reset()

        except RuntimeError or KeyboardInterrupt:
            print("entered init exception")
            for actor in world.get_actors():
                actor.destroy()
            pass

    def on_lane_invasion(self, event):
        self.problem = True 
        self.reset()
    
    def step(self, action):

        throttle = action[0]/10.0
        
        self.reached_waypoint_list.append(self.current_waypoint_index)

        if action[1] == 0:
            steer = 0.0
        elif action[1] <= 10:
            steer = -action[1]/10.0
        elif action[1]>10:
            steer = (action[1]-10.0)/10.0

        brake = action[2]/10.0

        if self.current_waypoint_index <10:
            throttle = 1.0
            brake = 0.0
            steer = 0.0

                        
        elif self.current_waypoint_index == 10:
            self.moving = True

        control = carla.VehicleControl(throttle=float(throttle), brake=float(brake), steer=float(steer)) #, steer=steer)
        self.vehicle.apply_control(control)



        obs = self.get_observation()

        
        
        # next_waypoint = self.waypoints[self.current_waypoint_index]
        # next_waypoint_location = carla.Location(x=next_waypoint[0], y=next_waypoint[1])
        # current_location = self.vehicle.get_location()
        # distance = current_location.distance(next_waypoint_location)

        reward = self.get_reward(obs)

        done = self.is_done(obs[3])


        

                                              

        #reward = self.get_reward(distance, control)
        #done = False

        info = {}

        sleep(0.1)

        return obs, reward, done, info
    
    
    def get_observation(self):
        state = []
        curr_yaw = self.vehicle.get_transform().rotation.yaw        
        
        for i in range(10):
            if self.current_waypoint_index + i < len(self.waypoints):
                waypoint = self.waypoints[self.current_waypoint_index + i]
                curr_vel = self.vehicle.get_velocity()
                curr_loc = self.vehicle.get_location()
                vx = curr_vel.x
                vy = curr_vel.y
                v = ((vx)**2+(vy)**2)**0.5
                distance = curr_loc.distance(carla.Location(x=waypoint[4],y=waypoint[5]))
                if distance >100:
                    distance =0
                state.extend([v, curr_yaw - waypoint[1], curr_yaw - waypoint[2], distance ])
                self.reached_waypoint_list.append(self.current_waypoint_index)
            else:
                state.extend([0, 0, 0, 0])
        return np.array(state)
    
    def get_reward(self, state):
        reward = -(np.linalg.norm(state[0] - self.waypoints[self.current_waypoint_index][0]) - (np.linalg.norm (state[1])) - (np.linalg.norm ( state[2])) - (np.linalg.norm (state[3])) + self.current_waypoint_index)
        return reward
    
    def close(self):
        self.vehicle.destroy()
        self.lane_invasion_sensor.destroy()
        self.collision_sensor.destroy()    

    def reset(self, seed=None, options=None):
        vel = carla.Vector3D()
        vel.x = 0
        vel.y = 0
        vel.z = 0
        self.vehicle.set_target_velocity(vel)
        transform = carla.Transform(carla.Location(x=-23.6,y=137.5),carla.Rotation(yaw=0))
        self.vehicle.set_transform(transform)
        self.current_waypoint_index = 0        
        self.moving = False
        #info = {}

        return self.get_observation()
    
    def render(self):
        pass

    def is_done(self, state):


        distance = np.abs(state)/100.0

        if distance < 0.4:  # If the vehicle is close to the next waypoint                                                                                        
            self.current_waypoint_index += 1  

        if len(self.reached_waypoint_list) >=21:
            if self.reached_waypoint_list[len(self.reached_waypoint_list)-1] == self.reached_waypoint_list[len(self.reached_waypoint_list) - 20] and self.moving:
                return  True
        if self.current_waypoint_index >= len(self.waypoints):
            return True
    
        
        return False


In [5]:
# Create a vectorized environment with SubprocVecEnv
num_envs=1
env = CarlaInstance(world = setupCarla(), waypoints = read_waypoints(CSV_FILE))

check_env(env)

# Instantiate the PPO agent
model = PPO("MlpPolicy", env, verbose=1, gamma=0.99, tensorboard_log="/home/kololhe/stuff/PPO_training_curve/", learning_rate=0.00099)

# Train the agent
model.learn(total_timesteps=1000000, progress_bar=True, reset_num_timesteps=50000)

# Save the trained model
model.save("carla_ppo")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to /home/kololhe/stuff/PPO_training_curve/PPO_10


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 75.6     |
|    ep_rew_mean     | -151     |
| time/              |          |
|    fps             | 9        |
|    iterations      | 1        |
|    time_elapsed    | 213      |
|    total_timesteps | 2048     |
---------------------------------
